### Problem 5: Enhanced Vigenere cipher

##### Steps to execute the following code:
1. Run the 1st Python code cell to compile the encrypt and decrypt functions
2. Run the 2nd Python code cell to test the implementation

In [1]:
import string
import random

# Returns key that is same length as given text (or +1 longer if multiple of 17)
def extendKey(plainText, key):
    new_key = key
    count = 0
    while len(new_key) != len(plainText):
        new_key += key[count]
        count += 1
        if count == len(key):
            count = 0
    if (len(plainText) % 17) == 0: # If key is multiple of 17, add another character
        new_key += key[count]
    return new_key

# Encrypts the given plaintext using the key a randomly generated string
def encryptText(plainText, key):
    randomstring = "" # String to insert in from of given plaintext

    # Generate random string
    for x in range(17):
        randomstring += random.choice(string.ascii_lowercase)

    new_plaintext = randomstring + plainText # Concatenation of randomly generated string and plaintext
    string_y = "" # String after 

    # Performance "enhancement"
    for x in range(len(new_plaintext)):
        if x < 17:
            string_y += new_plaintext[x]
        else:
            x_ascii_val = ord(new_plaintext[x])
            y_ascii_val = ord(string_y[x - 17])
            new_val = ((x_ascii_val + y_ascii_val - 97 - 97) % 26) + 97 # 97 is ASCII number for lowercase a
            new_char = chr(new_val)
            string_y += new_char

    # print("new_plaintext:\t" + new_plaintext) # for debugging, used to check if "enhancement" works
    # print("string y:\t" + string_y) # for debugging, used to check if "enhancement" works
    new_key = extendKey(new_plaintext, key)

    # Perform Vigenere Cipher
    cipherText = ""
    for x in range(len(new_plaintext)):
        cipherText += chr(((ord(string_y[x]) + ord(new_key[x]) - 97 - 97) % 26) + 97)

    print("ciphertext:\t" + cipherText)
    return cipherText

# Decrypts the ciphertext using the key
def decryptText(cipherText, key):
    new_key = extendKey(cipherText, key) # Extend key

    # Undo Vigenere Cipher
    uncipherText = ""

    for x in range(len(cipherText)):
        new_val = ord(cipherText[x]) - ord(new_key[x])
        if new_val < 0:
            new_val += 26
        new_val += 97
        uncipherText += chr(new_val)
    
    # print("undo vigenere:\t" + uncipherText) # for debugging: used to check if undoing the Vigenere Cipher worked

    # Undo "enhancement"
    plainText = "" # The plaintext message
    i = len(uncipherText) - 1 # Counter for loop (goes down from length of ciphertext to 16)

    # Stops at index 16 to ignore the randomly generated string
    while i > 16:
        new_val = ord(uncipherText[i]) - ord(uncipherText[i - 17])
        if new_val < 0:
            new_val += 26
        new_val += 97
        plainText += chr(new_val)
        i = i - 1

    # print("plaintext:\t" + plainText[::-1]) # for debugging: used to check plaintext
    # Plaintext is currently in reverse order
    return plainText[::-1]



In [2]:
key = "iamverystrongkey" # Key when performing Vigenere cipher
plainText = "hiwelcometochilishowmayitakeyourorder" # The plaintext to encrypt
print("key:\t" + key)
print("plaintext:\t" + plainText + "\n")
print("---------------------------\n")

for x in range(3):
    print("Iteration " + str(x+1) + ":\n")
    print("Encryption:")
    cipherText = encryptText(plainText, key)
    print("\nDecryption:")
    decryptedText = decryptText(cipherText, key)
    print("Decrypted text:\t" + decryptedText)
    print("\n---------------------------")


key:	iamverystrongkey
plaintext:	hiwelcometochilishowmayitakeyourorder

---------------------------

Iteration 1:

Encryption:
ciphertext:	fazxatuvuibijwwojeuekycciwyoduybgtxrjjfulzthlfcmcmwjen

Decryption:
Decrypted text:	hiwelcometochilishowmayitakeyourorder

---------------------------
Iteration 2:

Encryption:
ciphertext:	ymxkbulvaezhehxykxgcxzdticumcpjcquqdhwgvczzdjexxdwxcql

Decryption:
Decrypted text:	hiwelcometochilishowmayitakeyourorder

---------------------------
Iteration 3:

Encryption:
ciphertext:	begbbwangosskfjztaylozfiaiefnvhordtvqngxrrfncpdvpxgfiu

Decryption:
Decrypted text:	hiwelcometochilishowmayitakeyourorder

---------------------------


### Problem 9: RC4 implementation

##### *Steps to execute the following code:
1. Run the 1st Python code cell to compile the encrypt and decrypt functions
2. Run the 2nd Python code cell to test the implementation

In [3]:
import datetime

'''
Current date and time are used to generate a unique IV.
The function "datetime.datetime.now().timestamp()" generates a 17byte
output, which is then concatenated with itself and the first 32 bytes
are used as the IV for our cipher
'''

def IV():
    iv = datetime.datetime.now().timestamp()
    iv = (str(iv)+str(iv))[:32]
    return bytes(bytearray(iv, encoding="utf-8"))
    
def KSA(key):
    key.extend(current_iv)
    key_length = len(key)
    S = list(range(256))
    j = 0
    for i in range(256):
        j = (j + S[i] + key[i % key_length]) % 256
        S[i], S[j] = S[j], S[i]
    return S
    
def PRGA(S):
    i = 0
    j = 0
    while True:
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        S[i], S[j] = S[j], S[i]
        K = S[(S[i] + S[j]) % 256]
        yield K
        
def crypt(key, text):
    key = list(key)
    keystream = PRGA(KSA(key) )
    for i in range(3072):       # As recommended, the first 3072 bytes are dropped.
        next(keystream)
    res = []
    for c in text:
        val = ("%02X" % (c ^ next(keystream)))
        res.append(val)
    return res
    
def encrypt(pt, key):
    pt = list(pt)
    key = list(key)
    return crypt(key, pt)
    
def decrypt(ct, key):
    ct = bytes.fromhex(''.join(ct))
    res = crypt(key, ct)
    return bytes.fromhex("".join(res)).decode('utf-8')
    


In [6]:
# Test code
# A new IV and hence a different cipher text will be generated every time the code is executed.

current_iv = IV()
print("current IV: ", current_iv)

key = 'sixteenbytekeykey'
plaintext = 'PlainTextGoesHere'

plaintext = bytes(bytearray(plaintext, encoding="utf-8"))
key = bytes(bytearray(key, encoding="utf-8"))

ciphertext = encrypt(plaintext, key)
print('ciphertext:', ''.join(ciphertext))

decrypted = decrypt(ciphertext, key)
print('decrypted:', decrypted)

current IV:  b'1644553344.3377241644553344.3377'
ciphertext: EAC8860A98614B6A3F0106296278E744E1
decrypted: PlainTextGoesHere
